## Разбор решения

Будем считать что файл базы данных sqlite находится в корневом каталоге исполняемого файла программы(*.ipynb).

In [1]:
import sqlite3
import pandas as pd
import time
import os.path

conn = sqlite3.connect('de_test.db')
cur = conn.cursor() 

<b>Задаем через пользовательский ввод параметры отчета:</b>  
  
`date_from (str)` - дата начала формирования отчета   
`date_to (str)` - дата окончания формирования отчета  
`kkt_category (list)` - список категорий для отбора при формировании отчета   
`receipt_date (boolean)` - группировка по полю "receipt_date"  
`region (boolean)` - группировка по полю "region"  
`channel (boolean)` - группировка по полю "channel"  
`file_path (str)` - путь к файлу "product_names.csv"  

In [4]:
params = {}

# Period
while True:
    try:
        date_from = input('Enter "date_from" of the report in the following format: (yyyy-mm-dd): ')
        valid_date_from = time.strptime(date_from, '%Y-%m-%d')
        params['date_from'] = date_from
        break
    except ValueError:
        print('Incorrect date! Enter "date_from" of the report in the following format: (yyyy-mm-dd):')

while True:
    try:
        date_from = input('Enter "date_to" of the report in the following format: (yyyy-mm-dd): ')
        valid_date_from = time.strptime(date_from, '%Y-%m-%d')
        params['date_to'] = date_from
        break
    except ValueError:
        print('Incorrect date! Enter "date_to" of the report in the following format: (yyyy-mm-dd):')

# List of categories
while True:
    try:
        # будем считать что в наименовании категорий допускаются пробелы в начале и в конце строки, 
        # поэтому убирать пробелы не будем
        # так же проигнорируем случай если пользователь введет две запятые и категория окажется пустой строкой
        list_kkt_category = [x for x in input('Enter categories separated by commas:').split(',') if x]  
        params['kkt_category'] = list_kkt_category
        break
    except ValueError:
        print('Incorrect data entered! Enter categories separated by commas:')

# Group report by the following fields
while True:
    receipt_date = input('Is it necessary to group the report by field "receipt_date"?(Enter "Yes" or "No")')
    if receipt_date=='Yes':
        params['receipt_date'] = True
        break
    elif receipt_date=='No':
        params['receipt_date'] = False
        break
    else:
        print('Incorrect data entered! Enter "Yes" or "No".')
        continue

# region
while True:
    region = input('Is it necessary to group the report by field "region"?(Enter "Yes" or "No")')
    if region=='Yes':
        params['region'] = True
        break
    elif region=='No':
        params['region'] = False
        break
    else:
        print('Incorrect data entered! Enter "Yes" or "No".')
        continue
        
# channel
while True:
    region = input('Is it necessary to group the report by field "channel"?(Enter "Yes" or "No")')
    if region=='Yes':
        params['channel'] = True
        break
    elif region=='No':
        params['channel'] = False
        break
    else:
        print('Incorrect data entered! Enter "Yes" or "No".')
        continue
         
# file_path
while True:
    file_path = input('Enter the path to the file ("product_names.csv"):')
    if os.path.exists(file_path):
        params['file_path'] = file_path
        break
    else:
        print('The file does not exist at path "{file_path}"! Enter the path to an existing file.'.format(file_path=file_path))
        continue  


Enter "date_from" of the report in the following format: (yyyy-mm-dd): 2019-08-01
Enter "date_to" of the report in the following format: (yyyy-mm-dd): 2019-08-02
Enter categories separated by commas:
Is it necessary to group the report by field "receipt_date"?(Enter "Yes" or "No")Yes
Is it necessary to group the report by field "region"?(Enter "Yes" or "No")Yes
Is it necessary to group the report by field "channel"?(Enter "Yes" or "No")Yes
Enter the path to the file ("product_names.csv"):product_names.csv


Получаем следующие параметры:

In [2]:
# чтобы не вводить параметры, можно воспользоваться шаблоном параметров
params = {'date_from': '2019-08-01',
 'date_to': '2019-08-02',
 'kkt_category': [],
 'receipt_date': True,
 'region': True,
 'channel': True,
 'file_path': 'product_names.csv'} # путь к файлу необходимо заменить на свой

In [3]:
params

{'date_from': '2019-08-01',
 'date_to': '2019-08-02',
 'kkt_category': [],
 'receipt_date': True,
 'region': True,
 'channel': True,
 'file_path': 'product_names.csv'}

Создаем таблицу `"product_names"` из файла `"product_names.csv"`

In [4]:
df_product_names = pd.read_csv(params['file_path'])
df_product_names.to_sql('product_names', conn, if_exists='replace', index=False)

pd.read_sql('''
SELECT * FROM product_names LIMIT 5;
''', conn)

,brand,product_name_hash
0,marlboro,4d558fdb75ec0ef6412569b44c00707e66c8ab5a
1,marlboro,7186d1138ee3e8660f0abf94ff1824f0efad77ff
2,marlboro,9e133cf1ef7a72c71969d3cd12e5e408e080df6c
3,marlboro,5676fc528deabe3ed6aef693793a486c0a255803
4,marlboro,605a78ab5e99b8068ad35458cd16197fe7b1ec4c


Создаем таблицу `"filter_categories"`:  
Берем данные из таблицы `"kkt_categories"`.  

1) Если задан фильтр категорий, тогда подставляем категории по которым необходимо сделать отбор.  
  
2) Так же, независимо, задан ли фильтр или нет - всегда делаем отбор по пересечению дат (таблицы `"kkt_categories"`):  
  
`date_from` - Дата начала действия записи (включается)  
`date_till` - Дата начала действия записи (<b>не включается</b>)  
  
с параметрами формирования отчета (`"date_from"` и `"date_to"`) даты должны пересекаться. Таким образом мы выбираем актуальные кассы для заданного периода.

3) Берем записи таблицы только с максимальной версией классификации, разделяя по колонкам `"kkt_number"` и `"category"`.    

In [5]:
cur.execute('''
DROP TABLE IF EXISTS org_category_filter;
''')    

sql_query = '''

CREATE TEMP TABLE IF NOT EXISTS org_category_filter AS 

SELECT 
org_inn

FROM (

    SELECT 
    org_inn
    ,category
    ,kkt_number
    ,version
    ,kkt_categories.date_from
    ,kkt_categories.date_till
    ,MAX(date(kkt_categories.version)) OVER(PARTITION BY kkt_categories.kkt_number,kkt_categories.category) as max_version_date
    
    FROM kkt_categories
    JOIN kkt_info USING(kkt_number)
    
    WHERE 
    
    -- date_from - Дата начала действия записи (включается)
    -- date_till - Дата начала действия записи (не включается!!!)
    -- date('{date_from}') - Дата начала формирования отчета
    -- date('{date_to}') - Дата окончания формирования отчета
    
    -- если бы date_till - "включалась"(включительно была), тогда условие было бы следующим:
    -- (date_from<=date('{date_to}') and date_till>=date('{date_from}')) 
    -- но так как дата date_till - не включаительно, то условие было бы следующим:
    -- (date_from<=date('{date_to}') and date_till>date('{date_from}')) 
    
    (kkt_categories.date_from<=date('{date_to}') and kkt_categories.date_till>date('{date_from}'))
    
    {category_filter}
    
    
    ORDER BY kkt_number, kkt_categories.date_from
)

WHERE 
version=max_version_date

GROUP BY   
org_inn

'''.format(
    date_from=params['date_from']
    ,date_to=params['date_to']
    ,category_filter=' AND category IN ('+",".join("'"+x+"'" for x in params['kkt_category'])+')' 
    if params['kkt_category'] else ''
    )
    
cur.execute(sql_query)

pd.read_sql('''
SELECT *
FROM org_category_filter 
--ORDER BY kkt_number 
LIMIT 10''', conn)

,org_inn
0,0000000009
1,0000000011
2,0000000012
3,0000000013
4,0000000015
5,0000000017
6,0000000022
7,0000000030
8,0000000034
9,0000000035


In [6]:
cur.execute('''
DROP TABLE IF EXISTS org_category_filter;
''')    

sql_query = '''

CREATE TEMP TABLE IF NOT EXISTS org_category_filter AS 

SELECT 
org_inn

FROM (

    SELECT 
    org_inn
    ,category
    ,kkt_number
    ,version
    ,kkt_categories.date_from
    ,kkt_categories.date_till
    ,MAX(date(kkt_categories.version)) OVER(PARTITION BY kkt_categories.kkt_number,kkt_categories.category) as max_version_date
    
    FROM kkt_categories
    JOIN kkt_info USING(kkt_number)
    
    WHERE 
    
    -- date_from - Дата начала действия записи (включается)
    -- date_till - Дата начала действия записи (не включается!!!)
    -- date('{date_from}') - Дата начала формирования отчета
    -- date('{date_to}') - Дата окончания формирования отчета
    
    -- если бы date_till - "включалась"(включительно была), тогда условие было бы следующим:
    -- (date_from<=date('{date_to}') and date_till>=date('{date_from}')) 
    -- но так как дата date_till - не включаительно, то условие будет следующим:
    -- (date_from<=date('{date_to}') and date_till>date('{date_from}')) 
    
    (kkt_categories.date_from<=date('{date_to}') and kkt_categories.date_till>date('{date_from}'))
    {category_filter}
    
    ORDER BY kkt_number, kkt_categories.date_from
)

WHERE 
version=max_version_date

GROUP BY   
org_inn


'''.format(
    date_from=params['date_from']
    ,date_to=params['date_to']
    ,category_filter=' AND category IN ('+",".join("'"+x+"'" for x in params['kkt_category'])+')' 
    if params['kkt_category'] else ''
    )

cur.execute(sql_query)

pd.read_sql('''
SELECT *
FROM org_category_filter 
LIMIT 10''', conn)

,org_inn
0,0000000009
1,0000000011
2,0000000012
3,0000000013
4,0000000015
5,0000000017
6,0000000022
7,0000000030
8,0000000034
9,0000000035


Далее, создаем таблицу `"org_channel"` для вычисления поля `"channel"`.  
Получаем поля:  
`org_inn` - организация  
`channel` - является организация торговой сетью или нет. Учитывается:  
    - пересечение дат ("kkt_info.date_from", "kkt_info.date_till") и параметров отчета ("date_from","date_to")  
    - пересечение дат ("kkt_activity.receipt_date_min","kkt_activity.receipt_date_max") и параметров отчета ("date_from","date_to") 
`*shop_id` - поле отображено в примере ниже для наглядности чтобы убедиться в корректности результатов поля `channel`. В рабочем отчете это поле нужно удалить, так как в дальнейшем оно не используется в соединениях.

In [7]:
sql_query = ''' 

DROP TABLE IF EXISTS org_channel;

CREATE TEMP TABLE IF NOT EXISTS org_channel AS 


SELECT 
org_inn,
channel
FROM (

SELECT 
org_inn
,shop_id
,CASE 
    WHEN COUNT(shop_id) OVER(PARTITION BY org_inn) >= 3 
    THEN 'chain' 
    ELSE 'nonchain' 
    END AS channel

FROM (

        SELECT 
        * 
        FROM kkt_info 
        JOIN kkt_activity USING(kkt_number)
        WHERE     
        
          --  kkt_activity.receipt_date_min - Дата первого чека, полученного от кассы
          --  kkt_activity.receipt_date_max - Дата последнего чека, полученного от кассы
          --  date('{date_from}') - Дата начала формирования отчета 
          --  date('{date_to}') - Дата окончания формирования отчета 
         
          (receipt_date_min<=date('{date_to}') AND receipt_date_max>=date('{date_from}'))
        
        AND 
                  
          -- kkt_info.date_from - Дата начала действия записи (включается)
          -- kkt_info.date_till - Дата начала действия записи (не включается!!!)
          -- date('{date_from}') - Дата начала формирования отчета 
          -- date('{date_to}') - Дата окончания формирования отчета           
          -- так как kkt_info.date_till НЕ ВКЛЮЧИТЕЛЬНО значит условие будет следующее:
          -- kkt_info.date_from<=date('{date_to}') AND kkt_info.date_till>date('{date_from}')

            (kkt_info.date_from<=date('{date_to}') AND kkt_info.date_till>date('{date_from}'))
      
) 
        
GROUP BY 
org_inn
,shop_id
)
GROUP BY 
org_inn,
channel
      
'''.format(
    date_from=params['date_from']
    ,date_to=params['date_to']
    )

cur.executescript(sql_query)
    
    
pd.read_sql("SELECT * FROM org_channel LIMIT 20",conn)

,org_inn,channel
0,0000000004,nonchain
1,0000000007,nonchain
2,0000000008,nonchain
3,0000000009,nonchain
4,0000000012,nonchain
5,0000000013,nonchain
6,0000000014,nonchain
7,0000000015,nonchain
8,0000000017,chain
9,0000000018,nonchain


Если оставить колонку `"shop_id"`, то будет видно активные точки организаций: например, возьмем организацию `"0000000014"`, `"0000000015"` и `"0000000017"`.  
Видно что у организаций `"0000000014"` и `"0000000015"` только по одной активной точки продажи за период формирования отчета, и, следовательно значение `"channel"` = `"nonchain"`.  
А вот  у организации `"0000000017"` активных точек продажи за период формирования отчета 5, следовательно, значение `"channel"` = `"chain"`. Все верно.    


| org_inn | shop_id	| channel |
| ---|---|--- |
| 0000000014  | 1  | nonchain |
| 0000000015  | 2  | nonchain |
| 0000000017  | 1  | chain |
| 0000000017  | 2  | chain |
| 0000000017  | 3  | chain |
| 0000000017  | 4  | chain |
| 0000000017  | 5  | chain |



Теперь выполним итоговый запрос, соединив временные таблицы:

In [8]:
params['receipt_date'] = True
params['region'] = True
params['channel'] = True

def get_final_sql_query():
    return '''

    SELECT 
    *,
    SUM(sum) OVER(PARTITION BY brand) as sum_of_brand,
    SUM(sum) OVER() AS total_amount,
    ROUND(CAST(SUM(sum) OVER(PARTITION BY brand) AS FLOAT) / CAST(SUM(sum) OVER() AS FLOAT), 2) as percent_of_brand_sales
    FROM (
        SELECT       
            {selected_fields}
            
            product_names.brand,   
            SUM(sales.total_sum) as sum
        FROM sales
            JOIN kkt_info as t_info USING(kkt_number)
            JOIN product_names USING(product_name_hash) 
            JOIN org_channel USING(org_inn)
        WHERE 
            sales.product_name_hash IN (SELECT product_names.product_name_hash FROM product_names)
            AND (date(sales.receipt_date)>=date('{date_from}') AND date(sales.receipt_date)<=date('{date_to}'))
            {filter_category}
        GROUP BY 
            {selected_fields}
            brand
    )
    '''.format(date_from=params['date_from']
               ,date_to=params['date_to']
               ,filter_category = (
               ''' AND sales.org_inn IN (SELECT org_category_filter.org_inn 
                   FROM org_category_filter) ''' if params['kkt_category'] else '' )
              ,selected_fields=(
               ('receipt_date,' if params['receipt_date'] else '')+
               ('t_info.region,' if params['region'] else '')+
               ('org_channel.channel,' if params['channel'] else ''))
               )        

pd.read_sql(get_final_sql_query(),conn)

,receipt_date,region,channel,brand,sum,sum_of_brand,total_amount,percent_of_brand_sales
0,2019-08-01,Ленинградская область,chain,marlboro,4354,383844,1304304,0.29
1,2019-08-01,Ленинградская область,nonchain,marlboro,21584,383844,1304304,0.29
2,2019-08-01,Московская область,chain,marlboro,2054,383844,1304304,0.29
3,2019-08-01,Московская область,nonchain,marlboro,21619,383844,1304304,0.29
4,2019-08-01,Новосибирская область,chain,marlboro,9564,383844,1304304,0.29
5,2019-08-01,Новосибирская область,nonchain,marlboro,14906,383844,1304304,0.29
6,2019-08-01,Санкт-Петербург,chain,marlboro,6690,383844,1304304,0.29
7,2019-08-01,Санкт-Петербург,nonchain,marlboro,51806,383844,1304304,0.29
8,2019-08-01,Свердловская область,chain,marlboro,4076,383844,1304304,0.29
9,2019-08-01,Свердловская область,nonchain,marlboro,9195,383844,1304304,0.29


Попробуем сформировать отчет с различными вариантами группировок:

In [9]:
params['receipt_date'] = False
params['region'] = False
params['channel'] = False
pd.read_sql(get_final_sql_query(),conn)

,brand,sum,sum_of_brand,total_amount,percent_of_brand_sales
0,marlboro,383844,383844,1304304,0.29
1,parliament,920460,920460,1304304,0.71


In [10]:
params['receipt_date'] = True
params['region'] = False
params['channel'] = False
pd.read_sql(get_final_sql_query(),conn)

,receipt_date,brand,sum,sum_of_brand,total_amount,percent_of_brand_sales
0,2019-08-01,marlboro,187633,383844,1304304,0.29
1,2019-08-02,marlboro,196211,383844,1304304,0.29
2,2019-08-01,parliament,448686,920460,1304304,0.71
3,2019-08-02,parliament,471774,920460,1304304,0.71


In [11]:
params['receipt_date'] = True
params['region'] = True
params['channel'] = False
pd.read_sql(get_final_sql_query(),conn)

,receipt_date,region,brand,sum,sum_of_brand,total_amount,percent_of_brand_sales
0,2019-08-01,Ленинградская область,marlboro,25938,383844,1304304,0.29
1,2019-08-01,Московская область,marlboro,23673,383844,1304304,0.29
2,2019-08-01,Новосибирская область,marlboro,24470,383844,1304304,0.29
3,2019-08-01,Санкт-Петербург,marlboro,58496,383844,1304304,0.29
4,2019-08-01,Свердловская область,marlboro,13271,383844,1304304,0.29
5,2019-08-01,г. Москва,marlboro,41785,383844,1304304,0.29
6,2019-08-02,Ленинградская область,marlboro,38481,383844,1304304,0.29
7,2019-08-02,Московская область,marlboro,29575,383844,1304304,0.29
8,2019-08-02,Новосибирская область,marlboro,27298,383844,1304304,0.29
9,2019-08-02,Санкт-Петербург,marlboro,53297,383844,1304304,0.29


In [12]:
params['receipt_date'] = False
params['region'] = False
params['channel'] = True
pd.read_sql(get_final_sql_query(),conn)

,channel,brand,sum,sum_of_brand,total_amount,percent_of_brand_sales
0,chain,marlboro,60399,383844,1304304,0.29
1,nonchain,marlboro,323445,383844,1304304,0.29
2,chain,parliament,204145,920460,1304304,0.71
3,nonchain,parliament,716315,920460,1304304,0.71


Записываем результат отчета в файл `"report.csv"`

In [13]:
pd.read_sql(get_final_sql_query(),conn).to_csv('report.csv', index=False)

Теперь выведем три варианта отчета из третьего пункта задания:

In [14]:
params = {'date_from': '2019-08-01',
 'date_to': '2019-08-31',
 'kkt_category': [],
 'receipt_date': False,
 'region': False,
 'channel': False,
 'file_path': 'product_names.csv'}
pd.read_sql(get_final_sql_query(),conn).to_csv('report_1.csv', index=False)
pd.read_sql(get_final_sql_query(),conn)

,brand,sum,sum_of_brand,total_amount,percent_of_brand_sales
0,marlboro,6159519,6159519,18989072,0.32
1,parliament,12829553,12829553,18989072,0.68


In [15]:
params = {'date_from': '2019-08-01',
 'date_to': '2019-08-31',
 'kkt_category': [],
 'receipt_date': True,
 'region': True,
 'channel': True,
 'file_path': 'product_names.csv'}
pd.read_sql(get_final_sql_query(),conn).to_csv('report_2.csv', index=False)
pd.read_sql(get_final_sql_query(),conn)

,receipt_date,region,channel,brand,sum,sum_of_brand,total_amount,percent_of_brand_sales
0,2019-08-01,Ленинградская область,chain,marlboro,4354,6159519,18989072,0.32
1,2019-08-01,Ленинградская область,nonchain,marlboro,21584,6159519,18989072,0.32
2,2019-08-01,Московская область,chain,marlboro,2054,6159519,18989072,0.32
3,2019-08-01,Московская область,nonchain,marlboro,21619,6159519,18989072,0.32
4,2019-08-01,Новосибирская область,chain,marlboro,9564,6159519,18989072,0.32
...,...,...,...,...,...,...,...,...
737,2019-08-31,Санкт-Петербург,nonchain,parliament,51555,12829553,18989072,0.68
738,2019-08-31,Свердловская область,chain,parliament,19728,12829553,18989072,0.68
739,2019-08-31,Свердловская область,nonchain,parliament,50395,12829553,18989072,0.68
740,2019-08-31,г. Москва,chain,parliament,6404,12829553,18989072,0.68


In [16]:
params = {'date_from': '2019-08-05',
 'date_to': '2019-08-11',
 'kkt_category': ['FMCG'],
 'receipt_date': False,
 'region': True,
 'channel': False,
 'file_path': 'product_names.csv'}
pd.read_sql(get_final_sql_query(),conn).to_csv('report_3.csv', index=False)
pd.read_sql(get_final_sql_query(),conn)

,region,brand,sum,sum_of_brand,total_amount,percent_of_brand_sales
0,Ленинградская область,marlboro,194650,1339118,3821645,0.35
1,Московская область,marlboro,178800,1339118,3821645,0.35
2,Новосибирская область,marlboro,177037,1339118,3821645,0.35
3,Санкт-Петербург,marlboro,470649,1339118,3821645,0.35
4,Свердловская область,marlboro,75501,1339118,3821645,0.35
5,г. Москва,marlboro,242481,1339118,3821645,0.35
6,Ленинградская область,parliament,395642,2482527,3821645,0.65
7,Московская область,parliament,450938,2482527,3821645,0.65
8,Новосибирская область,parliament,452152,2482527,3821645,0.65
9,Санкт-Петербург,parliament,405396,2482527,3821645,0.65


In [17]:
conn.close()